<a href="https://colab.research.google.com/github/minsuzzz/kaggle_study/blob/main/kaggle_study_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1st level. Titanic: Machince Learning from Disaster

# **타이타닉 튜토리얼-1 Exploratory data analysis, visualizaation, machine learnig**

----

## Contents

  * 1. 데이터셋 확인
    * 1.1 Null data check
    * 1.2 Target label 확인
  * 2. Exploratory data analysis
    * 2.1 Pclass
    * 2.2 Sex
    * 2.3 Both Sex and Pclass
    * 2.4 Age
    * 2.5 Pclass, Sex, Age
    * 2.6 Embarked
    * 2.7 Family - SibSip(형제자매) + Parch(부모, 자녀)
    * 2.8 Cabin


* 만약 데이터 사이언스, 머신러닝 또는 캐글에서 어떤 것을 해야하는지 잘 모르는 newbie라면, 타이타닉을 하시는 게 가장 좋은 선택입니다.

* 타이타닉은 아시다시피, 사상 최대 해난사고로써, 1,500여명의 희생자가 생겼습니다.

* 우리는 이 튜토리얼에서 타이타닉에 탑승한 사람들의 신상정보를 활용하여, 승선한 사람들의 생존여부를 예측하는 모델을 생성할 것입니다.

* 본 튜토리얼에서는 여러 시각화 도구들(matplotlib, seaborn, plotly), 데이터 분석 도구(pandas, numpy), 머신 러닝 도구(sklearn)을 사용할 것입니다.

* 본 튜토리얼은 캐글에 있는 여러 커널들을 참조하여 만들었습니다. 본 튜토리얼을 공부하신 뒤에, 캐글 타이타닉 컴퍼티션에 존재하는 여러 다른 커널들을 더 공부하시면 됩니다.

* 본 튜터리얼은 파이썬 문법에 어려움이 없으셔야 수월할 것입니다. 여기서 사용하는 라이브러리들을 써본 경험이 있으면 좋겠지만, 경험이 없다하더라도 한 줄씩 천천히 적어나가시면 충분히 하실 수 있습니다.


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
sns.set(font_scale=2.5) #이 두줄은 본 필자가 항상 쓰는 방법입니다. matplotlib의 기본 scheme 말고 seaborn scheme을 세팅하고, 일일이 graph의 font size를 지정할 필요 없이 seaborn의 font_scale을 사용하면 편합니다.
import missingno as msno

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

앞으로 우리가 해야할 프로세스는 대략 아래와 같습니다. 

1. 데이터셋 확인 - 대부분의 캐글 데이터들은 잘 정제되어 있습니다. 하지만 가끔 null data가 존재합니다. 이를 확인하고, 향후 수정합니다.

2. 탐색적 데이터 분석(exploratory data anlysis) - 여러 feature 들을 개별적으로 분석하고 feature 들 간의 상관관계를 확인합니다. 여러 시각화 툴을 사용하여 insight를 얻습니다.

3. feature engineering - 모델을 세우기에 앞서, 모델의 성능을 높일 수 있도록 feature 들을 engineering 합니다. one-got encoing, class로 나누기, 구간으로 나누기, 텍스트 데이터 처리 등을 합니다.

4. model 만들기 - sklearn 을 사용해 모델을 만듭니다. 파이썬에서 머신러닝을 할 때는 sklearn 을 사용하면 수많은 알고리즘을 일관된 문법으로 사용할 수 있습니다. 물론 딥러닝을 위해 tensorflow, pytorch 등을 사용할 수 도 있습니다.

5. 모델 학습 및 예측 - trainset을 가지고 모델을 학습시킨 후, testset을 가지고 prediction 합니다.

6. 모델 평가 - 예측 성능이 원하는 수준인지 판단합니다. 풀려는 문제에 따라 모델을 평가하는 방식도 달라집니다. 학습된 모델이 어떤 것을 학습하였는지 확인해 봅니다.



## 1. Dataset 확인

* 파이썬에서 테이블화 된 데이터를 다루는 데 가장 최적화되어 있으며, 많이 쓰이는 라이브러리는 pandas 입니다.

* 우리는 pandas를 사용하여 데이터셋의 간단한 통계적 분석, 복잡한 처리들을 간단한 메소드를 사용하여 해낼 수 있습니다.

* 파이썬으로 데이터분석을 한다고 하면 반드시 능숙해져야 할 라이브러리이니, 여러 커널들을 공부하시면서 사용법에 익숙해지도록 반복 또 반복하시길 권장합니다.

* 캐글에서 데이터셋은 보통 train, testset 으로 나뉘어 있습니다.


In [6]:
!pip install gcsfs;

In [7]:

GCS_DS_PATH = "gs://kds-181a4deafa270c5ce4ace71ede22337472ee9dfc903a7326ed8770cb"
df_train = pd.read_csv(GCS_DS_PATH+'/train.csv')
df_train = pd.read_csv(GCS_DS_PATH+'/test.csv')
df_train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


* 우리가 다루는 문제에서 feature는 Pclass, Age, SibSp, Parch, Fare 이며, 예측하려는 target label은 Survived 입니다.

|**변수(feature, variable)**|정의|설명|타입|
|:------:|:---:|:---:|:---:|
|survival|생존여부|target label 임. 1, 0 으로 표현됨|integer|
|Pclass|티켓의 클래스|1 = 1st, 2 = 2nd, 3= 3rd 클래스로 나뉘며 categorical feature|integer|
|sex|성별|male, female 로 구분되며 binary|string|
|Age|나이|continuous|integer|
|sibSp|함께 탑승한 형제와 배우자의 수|quantitative|integer|
|parch|함께, 탑승한 부모, 아이의 수|quantittive|integer|
|ticket|티켓 번호|alphabat + integer|string|
|fare|탑승료|continuous|float|
|cabin|객실 번호|alphabat + integer|string|
|embared|탑승 항구|C = Cherbourg, Q = Queenstown, S=Southapton|string|

* pandas dataframe 에는 describe() 메소드가 있는데, 이를 쓰면 각 feature가 가진 통계치들을 반환해줍니다.


In [8]:
df_train.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


* 테이블에서 보다시피, PassenserID 숫자와 다른, 그러니까 null data가 존재하는 열(feature)가 있는 것 같습니다.
* 이를 좀 더 보기 편하도록 그래프로 시각화해서 살펴봅시다.


### 1.1 Null data check

In [11]:
for col in df_train.columns:
  msg = 'columns: {:>10}\t percent of NaN value: {:.2f}%'.format(col, 100 * (df_train[col].isnull().sum() / df_train[col].shape[0]))
  print(msg)

columns: PassengerId	 percent of NaN value: 0.00%
columns:     Pclass	 percent of NaN value: 0.00%
columns:       Name	 percent of NaN value: 0.00%
columns:        Sex	 percent of NaN value: 0.00%
columns:        Age	 percent of NaN value: 20.57%
columns:      SibSp	 percent of NaN value: 0.00%
columns:      Parch	 percent of NaN value: 0.00%
columns:     Ticket	 percent of NaN value: 0.00%
columns:       Fare	 percent of NaN value: 0.24%
columns:      Cabin	 percent of NaN value: 78.23%
columns:   Embarked	 percent of NaN value: 0.00%
